In [ ]:
import os, urllib
import numpy as np
from moviepy.editor import VideoFileClip
import cv2
import matplotlib.pyplot as plt
from IPython.display import Video

from sklearn.decomposition import PCA
from dca.dca import DynamicalComponentsAnalysis as DCA

# The Movie

I will be applying DCA on a scene from the adult animated science fiction action horror anthology film [*Predator: Killer of Killers*](https://en.wikipedia.org/wiki/Predator:_Killer_of_Killers), directed by Dan Trachtenberg. The entire film is 1.26 GB, so I extracted one of the scenes from the movie. You can read the plot of this scene [here](https://en.wikipedia.org/wiki/Predator:_Killer_of_Killers#The_Bullet). This data is extracted from Hulu.

# Load Movie

In [ ]:
scene = VideoFileClip("../data/predator_killer_of_killers_the_bullet.mp4")

## Extract Specific Clips

Possible test clips
- 0:25 to 0:31 (plane & car)
- 5:40 to 5:46 (moving alien projectile)
- 5:49 to 5:55 (flying airplanes #1)
- 6:00 to 6:05 (flying airplanes #2)
- 6:37 to 6:43 (radar)
- 10:24 to 10:29 (exploding airplane)
- 10:30 to 10:38 (flying airplanes #3)
- **13:30 to 13:34 (altitude meter)**
- 13:35 to 13:38 (alien ship)
- **14:26 to 14:31 (flying airplanes #4)**
- 16:27 to 16:30 (flying airplane in fire)
- **16:52 to 16:54 (flying airplane in fire #2)**
- **19:10 to 19:16 (flare gun)**

In [ ]:
clip1 = scene.subclip(810, 814) # from 13:30 to 13:34
clip1.write_videofile('../data/meter.mp4', codec='libx264')

clip2 = scene.subclip(866, 871) # from 14:26 to 14:31
clip2.write_videofile('../data/airplane.mp4', codec='libx264')

clip3 = scene.subclip(1012, 1014) # from 16:52 to 16:54
clip3.write_videofile('../data/projectiles.mp4', codec='libx264')

clip4 = scene.subclip(1150, 1156) # from 19:10 to 19:16
clip4.write_videofile('../data/flare.mp4', codec='libx264')

# Preprocess Clips into Frames

In [ ]:
def preprocess_clip(clip):
    frames = []
    while True:
        ret, frame = clip.read()
        if not ret:
            break
    
        # Resize and convery to grayscale
        frame = cv2.resize(frame, (32, 32))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray)
    clip.release()

    return np.array(frames)

In [ ]:
meter = cv2.VideoCapture('../data/meter.mp4')
airplane = cv2.VideoCapture('../data/airplane.mp4')
projectiles = cv2.VideoCapture('../data/projectiles.mp4')
flare = cv2.VideoCapture('../data/flare.mp4')

In [ ]:
meter_frames = preprocess_clip(meter)
X_meter = meter_frames.reshape(meter_frames.shape[0], -1) # (T=96 frames, 1024)

airplane_frames = preprocess_clip(airplane)
X_airplane = airplane_frames.reshape(airplane_frames.shape[0], -1) # (T=120 frames, 1024)

projectiles_frames = preprocess_clip(projectiles)
X_projectiles = projectiles_frames.reshape(projectiles_frames.shape[0], -1) # (T=48 frames, 1024)

flare_frames = preprocess_clip(flare)
X_flare = flare_frames.reshape(flare_frames.shape[0], -1) # (T=144 frames, 1024)

In [ ]:
X_meter_downsampled = X_meter[::4] # keep every fourth frame
X_meter_downsampled.shape

In [ ]:
X_airplane_downsampled = X_airplane[::5] # keep every fifth frame
X_airplane_downsampled.shape

In [ ]:
X_projectiles_downsampled = X_projectiles[::2] # keep every second frame
X_projectiles_downsampled.shape

In [ ]:
X_flare_downsampled = X_flare[::6] # keep every sixth frame
X_flare_downsampled.shape

# DCA <small>(vs. PCA on Clip 3/4)</small>

In [ ]:
dca = DCA(d=2, # extract 2 dynamical components
          T=3, # 2*T=6 frame window (safe given 72 frames total)
          verbose=True,
          block_toeplitz=True,
          rng_or_seed=10
         )

In [ ]:
pca = PCA(n_components=2)

## Clip 1 (meter.mp4)

In [ ]:
X_meter_dca = dca.fit_transform(X_meter_downsampled)

In [ ]:
plt.plot(X_meter_dca[:, 0], label="DC 1")
plt.plot(X_meter_dca[:, 1], label="DC 2")
plt.title("Dynamical Components over Time (Clip 1)")
plt.xlabel("Time (frame index)")
plt.ylabel("Component Value")
plt.legend()
plt.grid()
plt.savefig("../plots/meter_plot.png", dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
V = dca.coef_
V.shape

In [ ]:
dc1_map = V[:, 0].reshape((32, 32))
dc2_map = V[:, 1].reshape((32, 32))

### Reference Clip

In [ ]:
Video("../data/meter.mp4", embed=True, width=400)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(dc1_map, cmap='hot')
axes[0].set_title('Dynamical Component 1 – Pixel Contributions')
fig.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(dc2_map, cmap='hot')
axes[1].set_title('Dynamical Component 2 – Pixel Contributions')
fig.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.savefig("../plots/meter_dca.png", dpi=300, bbox_inches='tight')
plt.show();

## Clip 2 (airplane.mp4)

In [ ]:
X_airplane_dca = dca.fit_transform(X_airplane_downsampled)

In [ ]:
plt.plot(X_airplane_dca[:, 0], label="DC 1")
plt.plot(X_airplane_dca[:, 1], label="DC 2")
plt.title("Dynamical Components over Time (Clip 2)")
plt.xlabel("Time (frame index)")
plt.ylabel("Component Value")
plt.legend()
plt.grid()
plt.savefig("../plots/airplane_plot.png", dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
V = dca.coef_
V.shape

In [ ]:
dc1_map = V[:, 0].reshape((32, 32))
dc2_map = V[:, 1].reshape((32, 32))

### Reference Clip

In [ ]:
Video("../data/airplane.mp4", embed=True, width=400)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(dc1_map, cmap='hot')
axes[0].set_title('Dynamical Component 1 – Pixel Contributions')
fig.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(dc2_map, cmap='hot')
axes[1].set_title('Dynamical Component 2 – Pixel Contributions')
fig.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.savefig("../plots/airplane_dca.png", dpi=300, bbox_inches='tight')
plt.show();

## Clip 3 (projectiles.mp4)

In [ ]:
X_projectiles_dca = dca.fit_transform(X_projectiles_downsampled)

In [ ]:
plt.plot(X_projectiles_dca[:, 0], label="DC 1")
plt.plot(X_projectiles_dca[:, 1], label="DC 2")
plt.title("Dynamical Components over Time (Clip 3)")
plt.xlabel("Time (frame index)")
plt.ylabel("Component Value")
plt.legend()
plt.grid()
plt.savefig("../plots/projectiles_plot.png", dpi=300, bbox_inches='tight')
plt.show();

### Reference Clip

In [ ]:
Video("../data/projectiles.mp4", embed=True, width=400)

In [ ]:
V = dca.coef_
V.shape

In [ ]:
dc1_map = V[:, 0].reshape((32, 32))
dc2_map = V[:, 1].reshape((32, 32))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(dc1_map, cmap='hot')
axes[0].set_title('Dynamical Component 1 – Pixel Contributions')
fig.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(dc2_map, cmap='hot')
axes[1].set_title('Dynamical Component 2 – Pixel Contributions')
fig.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.savefig("../plots/projectiles_dca.png", dpi=300, bbox_inches='tight')
plt.show();

The first Dynamical Component (DC1) appears to highlight regions that are most predictive of upcoming visual changes:

- The high **positive component values** in the bottom right corner correspond to the motion of the alien spaceship, suggesting this region carries strong predictive information about future frames.
- The **negative component values** in the upper sky align with exploding projectiles and smoke, indicating these features may reflect past activity that influences upcoming dynamics.
- The **(near) zero component values** across the cloudy background imply that the sky contributes little to predicting what happens next. Consistent with its relatively static nature.

In [ ]:
X_projectiles_pca = pca.fit_transform(X_projectiles_downsampled)

In [ ]:
V = pca.components_
V.shape

In [ ]:
pc1_map = V[0].reshape(32, 32)
pc2_map = V[1].reshape(32, 32)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(pc1_map, cmap='hot')
axes[0].set_title('Principal Component 1')
fig.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(pc2_map, cmap='hot')
axes[1].set_title('Principal Component 2')
fig.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.savefig("../plots/projectiles_pca.png", dpi=300, bbox_inches='tight')
plt.show();

Principal component 1 (PC1) extracted from PCA seems to not capture any meaningful patterns, as the positive regions don't highlight an actual object in the clip. Principal component 2 (PC2) seems to capture the bright bullet projectiles in the sky. Either way, neither components do not capture the alien spaceship that moves across the clip, which DC1 was able to capture.

## Clip 4 (flare.mp4)

In [ ]:
X_flare_dca = dca.fit_transform(X_flare_downsampled)

In [ ]:
plt.plot(X_flare_dca[:, 0], label="DC 1")
plt.plot(X_flare_dca[:, 1], label="DC 2")
plt.title("Dynamical Components over Time (Clip 4)")
plt.xlabel("Time (frame index)")
plt.ylabel("Component Value")
plt.legend()
plt.grid()
plt.savefig("../plots/flare_plot.png", dpi=300, bbox_inches='tight')
plt.show();

### Reference Clip

In [ ]:
Video("../data/flare.mp4", embed=True, width=400)

In [ ]:
V = dca.coef_
V.shape

In [ ]:
dc1_map = V[:, 0].reshape((32, 32))
dc2_map = V[:, 1].reshape((32, 32))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(dc1_map, cmap='hot')
axes[0].set_title('Dynamical Component 1 – Pixel Contributions')
fig.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(dc2_map, cmap='hot')
axes[1].set_title('Dynamical Component 2 – Pixel Contributions')
fig.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.savefig("../plots/flare_dca.png", dpi=300, bbox_inches='tight')
plt.show();

The first Dynamical Component (DC1) extracted by DCA appears to isolate the core dynamics of the **flare's motion across the sky**.

- The **bright (positive) regions** in the component map align with the current location of the flare's bright core, suggesting this area is highly informative for predicting what comes next.
- The **dark (negative) regions** trace the flare’s fading trail, highlighting areas where the light has just passed and is dimming, encoding the immediate past.
- The **(near) zero values** across the static sky indicate these background regions contribute little to the temporal evolution, and are thus effectively ignored by DC1.

In [ ]:
X_flare_pca = pca.fit_transform(X_flare_downsampled)

In [ ]:
V = pca.components_
V.shape

In [ ]:
pc1_map = V[0].reshape(32, 32)
pc2_map = V[1].reshape(32, 32)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(pc1_map, cmap='hot')
axes[0].set_title('Principal Component 1')
fig.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(pc2_map, cmap='hot')
axes[1].set_title('Principal Component 2')
fig.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.savefig("../plots/flare_pca.png", dpi=300, bbox_inches='tight')
plt.show();

Principal component 1 (PC1) extracted from PCA seems to capture the background sky (all positive) instead of the center of the flare (negative), whereas DC1 extracted from DCA captured the movement of the flare. This shows DCA was able to capture the meaningful dynamics of the movement of the flare that gives the most information of what the next frame will look like.